In [ ]:
!pip install pandas_datareader

In [74]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import requests
from io import StringIO
import time
import random
import mysql.connector
from datetime import datetime

# 連接 MySQL 資料庫
db = mysql.connector.connect(host="138.3.214.21",
    user="cust-update", passwd="yfy0109!", db="CrawlerDB")
cursor = db.cursor()

In [76]:
def crawl_financial_Report(year,season,stock_number,url):
    
    form_data = {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'co_id':stock_number,
        'year': year,
        'season': season,
    }
    print(year,season,stock_number,url)
    
    r = requests.post(url,form_data)
    
    try:
        html_df = pd.read_html(r.text)[1].fillna("")   
        return html_df 
    except ValueError:
        print("No tables found.")

In [77]:
def insert_table(df,year,season,stock_number,company_name, report_name):
    seq = 1
    for index, row  in df.iterrows():  
        sql = "INSERT INTO mops_season_report (report_name, company_id, company_name,period_year,season,acct_name, this_year_amt, this_year_percent,last_year_amt, last_year_percent,creation_date,seq)  VALUES (%s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        if row['本季金額']=='' :  row['本季金額']=0 
        if row['本季金額(%)'] =='' :  row['本季金額(%)']=0  
        if row['去年同期金額'] =='' :  row['去年同期金額']=0  
        if row['去年同期金額(%)'] =='' :  row['去年同期金額(%)']=0 
        now = datetime.now()
        val = (report_name,stock_number, company_name,year,season, row['會計項目'] , row['本季金額'], row['本季金額(%)'], row['去年同期金額'], row['去年同期金額(%)'],now,seq)
        cursor.execute(sql, val)
        seq += 1
        db.commit()     


In [121]:
# 連接 MySQL 資料庫
db = mysql.connector.connect(host="138.3.214.21",
    user="cust-update", passwd="yfy0109!", db="CrawlerDB")
cursor = db.cursor()
year = 112
season = 3
# 網站連線資料
BalanceSheetURL = "https://mops.twse.com.tw/mops/web/ajax_t164sb03";      # 資產負債表
ProfitAndLoseURL = "https://mops.twse.com.tw/mops/web/ajax_t164sb04";    # 損益表
CashFlowStatementURL = "https://mops.twse.com.tw/mops/web/ajax_t164sb05"; # 現金流量表
# 讀取全部公司代碼
comp_sql = 'SELECT distinct(company_id) , company_name FROM mops_monthly_report WHERE company_id NOT IN (SELECT DISTINCT company_id FROM  mops_season_report ) ORDER BY company_id'
cursor.execute(comp_sql)
comp_list = cursor.fetchall() 
#print(comp_list)

In [ ]:
year = 112
season = 1
df3= crawl_financial_Report(year,season,4736,CashFlowStatementURL)
df3= df3.iloc[:, [0,1,2,3,4]]
if df3 is  None : print('None')
df3
#df3.columns = [('會計項目'),('本季金額'), ('去年同期金額'), ('本季金額(%)'), ('去年同期金額(%)')]
#insert_table(df3,year,season,1454,'台富', 'CashFlowStatement')

In [149]:
#year = 112
#season = 1
#df31= crawl_financial_Report(year,season,4736,ProfitAndLoseURL)
df31.index
cols = len(df31.axes[1])
cols
#df3= df3.iloc[:, [0,1,2,3,4]]
#df3.columns = [('會計項目'),('本季金額'), ('去年同期金額'), ('本季金額(%)'), ('去年同期金額(%)')]
#insert_table(df3,year,season,1454,'台富', 'ProfitAndLose')

6

In [150]:
#year = 112
#season = 1
#df3= crawl_financial_Report(year,season,1435,ProfitAndLoseURL)
df3.index
cols = len(df3.axes[1])
cols
#df3= df3.iloc[:, [0,1,2,3,4]]
#df3.columns = [('會計項目'),('本季金額'), ('去年同期金額'), ('本季金額(%)'), ('去年同期金額(%)')]
#insert_table(df3,year,season,1454,'台富', 'ProfitAndLose')

1

In [112]:
#讀取公司代碼依序爬取財務資料寫入資料庫
year = 112
season = 1
for id in comp_list:
    stock_number = id[0]
    company_name = id[1]
    df1 = crawl_financial_Report(year,season,stock_number,BalanceSheetURL)
    df1= df1.iloc[:, [0,1,2,3,4]]
    df1.columns = [('會計項目'),('本季金額'), ('本季金額(%)'), ('去年同期金額'),('去年同期金額(%)')]
    insert_table(df1,year,season,stock_number,company_name, 'BalanceSheet')
    df2 = crawl_financial_Report(year,season,stock_number,ProfitAndLoseURL)
    df2= df2.iloc[:, [0,1,2,3,4]]
    df2.columns = [('會計項目'),('本季金額'), ('本季金額(%)'), ('去年同期金額'),('去年同期金額(%)')]
    insert_table(df2,year,season,stock_number,company_name, 'ProfitAndLose')
    df3 = crawl_financial_Report(year,season,stock_number,CashFlowStatementURL)
    df3= df3.iloc[:, [0,1,2,3,4]]
    df3.columns = [('會計項目'),('本季金額'), ('去年同期金額'), ('本季金額(%)'), ('去年同期金額(%)')]
    insert_table(df3,year,season,stock_number,company_name, 'CashFlowStatement')
    print('ok')
    time.sleep(random.randint(1,10))#隨機停等秒數

112 1 1101 https://mops.twse.com.tw/mops/web/ajax_t164sb03
112 1 1101 https://mops.twse.com.tw/mops/web/ajax_t164sb04
112 1 1101 https://mops.twse.com.tw/mops/web/ajax_t164sb05
ok
112 1 1102 https://mops.twse.com.tw/mops/web/ajax_t164sb03
112 1 1102 https://mops.twse.com.tw/mops/web/ajax_t164sb04
112 1 1102 https://mops.twse.com.tw/mops/web/ajax_t164sb05
ok
112 1 1103 https://mops.twse.com.tw/mops/web/ajax_t164sb03
112 1 1103 https://mops.twse.com.tw/mops/web/ajax_t164sb04
112 1 1103 https://mops.twse.com.tw/mops/web/ajax_t164sb05
ok
112 1 1104 https://mops.twse.com.tw/mops/web/ajax_t164sb03
112 1 1104 https://mops.twse.com.tw/mops/web/ajax_t164sb04
112 1 1104 https://mops.twse.com.tw/mops/web/ajax_t164sb05
ok
112 1 1108 https://mops.twse.com.tw/mops/web/ajax_t164sb03
112 1 1108 https://mops.twse.com.tw/mops/web/ajax_t164sb04
112 1 1108 https://mops.twse.com.tw/mops/web/ajax_t164sb05
ok
112 1 1109 https://mops.twse.com.tw/mops/web/ajax_t164sb03
112 1 1109 https://mops.twse.com.tw/mops/

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None))